Hate Speech Filtering

Using Tf-Idf and BoW to build SVM model

Linear model? Or RBF model?

RACIST texts classified with '1'
NON-RACIST texts classified with '0'


In [27]:
# Libraries


import nltk
# nltk.download('wordnet')
import re 

import pandas as pd
import numpy as np

import nltk
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.svm import SVC
import matplotlib.pyplot as plt

# NLTK Stemmer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Logistic Regression
lreg = LogisticRegression()
from sklearn.metrics import f1_score

In [ ]:
# Read data pandas, keep raw copy of data
train_RAW = pd.read_csv('train.csv')
test_RAW = pd.read_csv('test.csv')
train = train_RAW
test = test_RAW

TF-IDF (term frequency-inverse document frequency) is a statistical measure that evaluates how relevant a word is to a document in a collection of documents.
This is done by multiplying two metrics: how many times a word appears in a document, and the inverse document frequency of the word across a set of documents.

Ref: 
    https://monkeylearn.com/blog/what-is-tf-idf/#:~:text=TF%2DIDF%20(term%20frequency%2D,across%20a%20set%20of%20documents


Doc:
    https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [ ]:
# TF-IDF freq
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.90, 
    min_df=2, 
    max_features=6000, 
    stop_words='english'
    )

Bag of words is a Natural Language Processing technique of text modelling. 
In technical terms, we can say that it is a method of feature extraction with text data.

Ref: 
    https://www.mygreatlearning.com/blog/bag-of-words/


Doc:
    https://scikit-learn.org/stable/modules/feature_extraction.html

In [ ]:
# Bag Of Words Model
bow_vectorizer = CountVectorizer(
    max_df=0.90, 
    min_df=2, 
    max_features=6000, 
    stop_words='english'
    )

Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

Ref/Doc:
    https://scikit-learn.org/stable/modules/svm.html#:~:text=Support%20vector%20machines%20(SVMs)%20are,than%20the%20number%20of%20samples

In [30]:
# Combine training data with test data
allTweets = train.append(test, ignore_index=True, sort=False)

allTweets.tail()

/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_32898/1233045329.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  allTweets = train.append(test, ignore_index=True, sort=False)


,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [ ]:
def fnc_clean_tweet(tweet_txt, pattern):
    r = re.findall(pattern, tweet_txt)
    for i in r:
        tweet_txt = re.sub(i, '', tweet_txt)  # Substitute pattern with blank

    return tweet_txt

In [ ]:
# Clean text 
# Add new column 'tidy_tweet'

# Clean user handles (@user_name)
allTweets['tidy_tweet'] = np.vectorize(fnc_clean_tweet)(
    allTweets['tweet'], "@[\w]*") 

# Clean non-alphabets
allTweets['tidy_tweet'] = allTweets['tidy_tweet'].str.replace(
    "[^a-zA-Z#]", " ") 

allTweets.head()

In [ ]:
# Tokenization
tokenized_tweet = allTweets['tidy_tweet'].apply(lambda x: x.split())

tokenized_tweet.head()

In [ ]:
# # Stemming 
# # TODO: need to be worked on more, words not stemmed properly

# tokenized_tweet = tokenized_tweet.apply(
#     lambda x: [stemmer.stem(i) for i in x])  # stemming


# # Combine tokens back together
# for i in range(len(tokenized_tweet)):
#     tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

# # Replace with stemmed 
# allTweets['tidy_tweet'] = tokenized_tweet


# allTweets.head()

In [ ]:
# Using lemmatizer instead of stemmer
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('omw-1.4')
nltk.download('wordnet')
tokenized_tweet = tokenized_tweet.apply(
    lambda x: [lemmatizer.lemmatize(i) for i in x])  

# Combine tokens back together
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

# Replace with lemmatized
allTweets['tidy_tweet'] = tokenized_tweet

allTweets.head()


In [ ]:
# Feature extraction - Bag-of-Words [sklearn’s CountVectorizer]
# ------------------------------------------------------------------------------------------------------------------

# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(allTweets['tidy_tweet'])

print(bow)

In [ ]:

# TF-IDF Features - Looks at frequency of occurence for terms/importance of the term

# TF = (Number of times term t appears in a document)/(Number of terms in the document)
# IDF = log(N/n), where, N is the number of documents and n is the number of documents a term t has appeared in.
# 
# TF-IDF = TF*IDF

# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(allTweets['tidy_tweet'])

print(tfidf)


In [ ]:
train_bow = bow[:31962, :]
test_bow = bow[31962:, :]

# splitting data into training and validation set



# xtrain_bow training dataset
# xvalid_bow validation for training
# ytrain label vector
# yvalid validation label vector
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(
    train_bow, train['label'], test_size=0.1)

# train_bow feature matrix
# train['label'] label vector
# random_state shuffles data before split into training and testing
# test_size percentage of data gets tested on (0.9 training)


# print(xtrain_bow)
# print(xvalid_bow)
print(ytrain)
# print(yvalid)

# lreg = LogisticRegression()
# lreg.fit(xtrain_bow, ytrain)    
# train using training dataset and label vector

In [ ]:
# tfidf feature matrix
train_tfidf = tfidf[:31962, :]
test_tfidf = tfidf[31962:, :]


# ytrain.index index of axis labels 
xtrain_tfidf = train_tfidf[ytrain.index]
xvalid_tfidf = train_tfidf[yvalid.index]

# extract label vectors from the feature matrix via matching index

# print(xtrain_tfidf)


# fit on logistic regression
# lreg.fit(xtrain_tfidf, ytrain) 

In [ ]:
# SVM
svm = SVC(kernel='rbf')

SVM tutorial: https://www.youtube.com/watch?v=FB5EdxAGxQg

In [ ]:
# TODO: understand how svm model is built using BoW and TF-IDF

svm.fit(xtrain_bow, ytrain)        # Build using bow
# accuracy test
svm.score(xvalid_bow, yvalid)       

In [ ]:
svm.fit(xtrain_tfidf, ytrain)      # Build using TD-IDF
# svm.predict()

svm.score(xvalid_tfidf, yvalid)


In [29]:
# plt.figure(figsize=(16,5))
# plt.plot(xvalid_tfidf, yvalid)

lreg.fit(xtrain_tfidf, ytrain)                   # Train model

print(xtrain_tfidf)

prediction = lreg.predict_proba(xvalid_tfidf)    # Predict validation set
prediction_int = prediction[:,1] >= 0.3
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int)


  (0, 343)	0.29188034974046045
  (0, 631)	0.33999969477857217
  (0, 2422)	0.4204121982725948
  (0, 3520)	0.24578186116628126
  (0, 4109)	0.31876934796752654
  (0, 4292)	0.3135049280930342
  (0, 5387)	0.30282436814586644
  (0, 5392)	0.3137986577923866
  (0, 5527)	0.3341725991303641
  (0, 5732)	0.2455772190657296
  (1, 505)	0.5843222589842813
  (1, 3003)	0.5163961965879316
  (1, 3754)	0.5431965840627105
  (1, 5394)	0.3111911580807328
  (2, 827)	0.4014224757585932
  (2, 2539)	0.32214155957109086
  (2, 2627)	0.41877109411793484
  (2, 3858)	0.4392838032267027
  (2, 3905)	0.48976569022567723
  (2, 4735)	0.35619501847255536
  (3, 445)	0.47997076441169745
  (3, 3998)	0.7246307675279473
  (3, 5348)	0.49450815570818346
  (4, 481)	0.5485282804215882
  (4, 1313)	0.4812945015641103
  :	:
  (28761, 2971)	0.4387384483244518
  (28761, 3645)	0.24888291766138218
  (28761, 4019)	0.5174577923851978
  (28761, 4320)	0.5003931349250015
  (28761, 4623)	0.4768752571459091
  (28762, 781)	0.3570239743490468
  (2

/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_32898/845311533.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  prediction_int = prediction_int.astype(np.int)


0.652054794520548